In [2]:
#### To run the functions in another notebook file, use the following:\
# %run Functions.ipynb     #this is were my function was stored\
# function(df)                    #then simply run the function


# Training and testing data

In [2]:
def TrainTestSplit(df,separate = 1,latest_year = 2017):
    print('Train test split')
    clientID = df.ClientID.unique()
    if separate == 1:
        # separating the living situation and the exit destination into categories
        goal = [10,11]
        closer = [3,19,20,21,22,23,25,26,28,31]
        trans = [1,2,4,5,6,12,13,14,15,18,27,29]
        no_progress = [7,16]
        hard_to_judge = [8,9,17,24,30,99]
        # Separating out a sample of data
        sample_data = pd.DataFrame()
        for i in range(len(clientID)):
            temp = df[df['ClientID']==clientID[i]]
            sample_data = sample_data.append(temp)
        sample_data = sample_data.reset_index(drop=True)
        # sample_data.to_csv(rpath+"//20220329_Sample data of 10 clients.csv")

        # Separating out the data with ultimate goal, closer to exit, and transitional phase
        sample_client = sample_data.ClientID.unique()
        data = pd.DataFrame()
        for i in range(len(sample_client)):
            temp = sample_data[sample_data['ClientID']==sample_client[i]].reset_index(drop=True)
            if temp.Destination.iloc[-1] in goal or temp.Destination.iloc[-1] in closer or temp.Destination.iloc[-1] in trans:
                data = data.append(temp)
    else:
        data = df
    
    
    #Determining train data
    print("Preparing train data")
    client = data.ClientID.unique()
    trainData = pd.DataFrame()

    for i in range(len(client)):
        temp = data[data['ClientID']==client[i]].reset_index(drop=True)
        temp = temp[temp['EntryDate'].dt.year<latest_year]
        if len(temp)==0:
            continue
        trainData = trainData.append(temp)


    # Determining historical data and queries
    print("Preparing test data")
    testData = pd.DataFrame()
    for i in range(len(client)):
        temp = data[data['ClientID']==client[i]].reset_index(drop=True)
        temp = temp[temp['EntryDate'].dt.year>=latest_year]
        if len(temp)==0:
            continue
        testData = testData.append(temp)


    # historical data: 80% of TestData
    # queries: 20% of TestData

    test_client_list = testData.ClientID.unique()

    historicalPercent = 0.8
    queryPercent = 1 - historicalPercent
    print("Preparing historical and query data")
    historicalClient = pd.DataFrame(test_client_list[0:int(historicalPercent*len(test_client_list))])
    queryClient = pd.DataFrame(test_client_list[int(historicalPercent*len(test_client_list))::])

    
    return trainData, testData, historicalClient, queryClient



        

# Transition graphs

In [9]:
def transition_graph_snapshot(trainData):
    print('Preparing snapshot graph')
    # Categorizing the training data using the effective length
    trainClient = trainData.ClientID.unique()
    trainData_lengthSeparated = pd.DataFrame()
    for i in range(len(trainClient)):
        temp = trainData[trainData['ClientID']==trainClient[i]].reset_index(drop=True)
        effective_length = len(temp['ProjectType'].unique())
        temp['effective length'] = [effective_length for j in range(len(temp))]
        trainData_lengthSeparated = trainData_lengthSeparated.append(temp)
    #Building transition graph for each effective length
    effective_length_unique = trainData_lengthSeparated['effective length'].unique()
    normalized_tm_combined = pd.DataFrame()
    for n in range(len(effective_length_unique)):
        print(effective_length_unique[n])
        temp1 = trainData_lengthSeparated[trainData_lengthSeparated['effective length']==effective_length_unique[n]]
        temp_clientID = temp1['ClientID'].unique()

        # Using dynamic programming style to determine the edges at each step considering the hops
        DP_edge_df = pd.DataFrame()
        for i in range(len(temp_clientID)):
            temp = temp1[temp1['ClientID']==temp_clientID[i]]
            project_list = list(temp.ProjectType)+[-3]
        #     print(project_list)
            DP = pd.DataFrame(index=range(len(project_list)),columns = [k for k in range(1,len(project_list)+1)])
            closed = []
            for d in range(1,len(project_list)+1):
                for a in range(len(project_list)):
                    for b in range(len(project_list)):
                        if (project_list[a],project_list[b],d) not in closed:
                            if (b-a) == d:
                                DP.at[a,d] = (project_list[a],project_list[b])
                                DP.at[a,'Position'] = a+1
                                closed = closed + [(project_list[a],project_list[b],d)]
            DP = DP.dropna(how = 'all',axis = 0)
            DP_edge_df = DP_edge_df.append(DP)

        DP_edge_df = DP_edge_df.dropna(how = 'all',axis = 1)  

        # Determining unique edges and creating a matrix for each edge
        edge_list = []
        for i in DP_edge_df.columns:
            if i!='Position':
                temp = DP_edge_df[i].dropna().unique()
                edge_list = edge_list + list(temp)

        edge_list_df = pd.DataFrame()
        edge_list_df['edges'] = [edge_list[i] for i in range(len(edge_list))]
        edge_list_unique = edge_list_df['edges'].unique()
        hop_unique = DP_edge_df['Position'].unique()

        # Counting the edges in each step
        edge_count = pd.DataFrame()
        for k in hop_unique:
            for i in edge_list_unique:
                temp = pd.DataFrame(index = edge_list_unique, columns = list(DP_edge_df.columns))
                for j in DP_edge_df.columns:
                    if j!='Position':
                        temp.at[i,j] = len(DP_edge_df[(DP_edge_df[j]==i) & (DP_edge_df['Position']==k)])            
                temp.at[i,'Position'] = k 
                temp = temp.dropna()
                edge_count = edge_count.append(temp)

        edge_count = edge_count.reset_index()
        edge_count = edge_count.rename(columns = {'index':'edges'})

        edge_count_matrix = edge_count.groupby(['edges','Position']).sum()

        # adding weights based on the hop and position
        alpha = 0.5
        weighted_edge_matrix = pd.DataFrame()
        for i in edge_list_unique:
            temp = edge_count_matrix.loc[i]
            weighted_edge = pd.DataFrame(index = temp.index,columns = temp.columns)
            for j in range(len(temp.columns)):#Hops
                for k in range(len(temp.index)):#Position
                    weighted_edge.at[temp.index[k],temp.columns[j]] = (alpha**(j+k))  * (temp.at[temp.index[k],temp.columns[j]])
        #             if weighted_edge.at[temp.index[k],temp.columns[j]]:
        #                 print(j,k)
        #                 print(weighted_edge.at[temp.index[k],temp.columns[j]],(alpha**(j+k)),(temp.at[temp.index[k],temp.columns[j]]))
            weighted_edge['edges'] = [i for a in range(len(weighted_edge))]
            weighted_edge_matrix = weighted_edge_matrix.append(weighted_edge)

        # summing the elements of the matrix to get the total weight
        final_weights = pd.DataFrame(index=edge_list_unique,columns=['weight'])
        for i in edge_list_unique:
            final_weights.at[i,'weight'] = weighted_edge_matrix[weighted_edge_matrix['edges']==i].drop('edges',axis=1).sum().sum()    



        # summing the elements of the matrix to get the total number of times the edges appear in the data
        final_weights['total'] = np.nan
        for i in edge_list_unique:
            final_weights.at[i,'total'] = int(edge_count_matrix.loc[i].sum().sum())   

        final_weights['normalized weight'] = final_weights['weight']/final_weights['total']




        # Building the transition matrix
        transition_matrix = pd.DataFrame(index = list(np.sort(trainData['ProjectType'].unique()))+[-3], columns = list(np.sort(trainData['ProjectType'].unique()))+[-3])
        for i in final_weights.index:
            transition_matrix.at[i[0],i[1]] = final_weights.at[i,'normalized weight']

        # Normalizing the transition matrix
        normalized_tm = pd.DataFrame()
        for i in transition_matrix.index:
            temp = transition_matrix.loc[[i]]
            denum = temp.sum(axis=1)[i]
            if temp.isnull().all(axis=1).loc[i]:
                normalized_tm = normalized_tm.append(temp)
                continue
            temp = temp/denum
            normalized_tm = normalized_tm.append(temp)

        normalized_tm['effective length'] = [effective_length_unique[n] for k in range(len(normalized_tm))]
        normalized_tm = normalized_tm.reset_index()
        normalized_tm_combined = normalized_tm_combined.append(normalized_tm)

    return normalized_tm_combined


def transition_graph_aggregate(trainData):
    print('Preparing aggregate graph ')
    # Generating the transition graph for the project type of train data
    # Unique Edge list
    edge_list = []
    trainClient = trainData.ClientID.unique()
    for i in range(len(trainClient)):
        temp = trainData[trainData['ClientID']==trainClient[i]]
        trajectory = temp.ProjectType.values
        for j in range(len(trajectory)):
            if j == len(trajectory)-1:
                edge_list = edge_list + [(trajectory[j],-3)] 
                break
            edge_list = edge_list + [(trajectory[j],trajectory[j+1])]

    edge_list = pd.DataFrame({'edge_list':edge_list})
    edge_unique = edge_list['edge_list'].unique()

    # Calculating the number of times a node appear in the outgoing end of the edge
    graph_nodes = pd.Series([edge_unique[i][0] for i in range(len(edge_unique))])
    graph_nodes_count = pd.DataFrame(index = graph_nodes.unique(),columns = ['count'])
    nodeOfOutgoingEdge = pd.Series([edge_list['edge_list'][i][0] for i in range(len(edge_list))])
    for i in range(len(graph_nodes_count)):
        n = len(nodeOfOutgoingEdge[nodeOfOutgoingEdge==graph_nodes_count.index[i]])
        graph_nodes_count.at[graph_nodes_count.index[i],'count'] = n

        # Calculating the transition probability of the edges in the data
    edge_probability = pd.DataFrame(columns = ['probability'],index=edge_unique)
    for i in range(len(edge_unique)):
        n = len(edge_list[edge_list['edge_list']==edge_unique[i]])
        outgoing_node_count = graph_nodes_count.loc[edge_unique[i][0]]['count']
        edge_probability.at[edge_unique[i],'probability'] = n/outgoing_node_count

    # Generating the transition matrix (rows:starting point of an edge, columns:ending point of an edge)
    # sorted unique project types in the edges
    ind = list(np.sort(trainData['ProjectType'].unique()))+[-3]
    col = list(np.sort(trainData['ProjectType'].unique()))+[-3]
    transition_matrix = pd.DataFrame(columns = col,index=ind)
    for i in range(len(edge_probability)):
        edge = edge_probability.index[i]
        transition_matrix.at[edge[0],edge[1]] = edge_probability.iloc[i].values[0]
    transition_matrix = transition_matrix.reset_index()    
    return transition_matrix


def GraphConstruction(transition_matrix,method):
    print('Constructing the graph in networkx')
    if method == 'snapshot':
        # Developing the transition graphs for each effective length in networkx
        normalized_tm_combined = transition_matrix
        effective_length_unique = normalized_tm_combined['effective length'].unique()
        projectType = normalized_tm_combined['index'].unique()

        G_list = {}
        Dg_list = {}
        shortest_path_length_list = {}
        for n in range(len(effective_length_unique)):
            print(n)
            G = nx.DiGraph()
            none = [G.add_node(k) for k in projectType]
            normalized_tm = normalized_tm_combined[normalized_tm_combined['effective length']==effective_length_unique[n]]
            normalized_tm = normalized_tm.set_index('index')
            normalized_tm = normalized_tm.drop('effective length',axis=1)
            normalized_tm.columns = pd.to_numeric(normalized_tm.columns)
            for i in range(len(normalized_tm.index)):
                nodes = normalized_tm.loc[normalized_tm.index[i]][normalized_tm.loc[normalized_tm.index[i]].notna()].index.values
                for k in nodes:
                    G.add_edge(normalized_tm.index[i],k,weight=normalized_tm.loc[normalized_tm.index[i]][k])  
#             plt.figure(figsize=(5,5))
            pos = nx.shell_layout(G)
            degree = dict(G.degree)
            edges = G.edges()
            weights = [G[u][v]['weight'] for u,v in edges]
#             nx.draw(G, pos, with_labels = True,node_size=[v * 100 for v in degree.values()], node_color="red")


            shortest_path_length_matrix = pd.DataFrame(index = G.nodes, columns = G.nodes)

            for i in G.nodes:
                for j in G.nodes:
                    if np.isnan(normalized_tm.loc[i][j]):
                        continue
                    if i==j:
                        shortest_path_length_matrix.loc[i][j] = G[i][j]['weight']
                    elif i!=j:
                        shortest_path_length_matrix.loc[i][j] = nx.shortest_path_length(G,i,j,weight = 'weight')


            shortest_path_length_list[effective_length_unique[n]] = shortest_path_length_matrix
            G_list[effective_length_unique[n]] = G
            Dg_list[effective_length_unique[n]] = shortest_path_length_matrix.max().max()
        return shortest_path_length_list,G_list,Dg_list
    elif method == 'aggregate':
        normalized_tm = transition_matrix
        # Developing the transition graphs for each effective length in networkx
        projectType = normalized_tm['index'].unique()
        G = nx.DiGraph()
        none = [G.add_node(k) for k in projectType]
        normalized_tm = normalized_tm.set_index('index')
        normalized_tm.columns = pd.to_numeric(normalized_tm.columns)
        for i in range(len(normalized_tm.index)):
            nodes = normalized_tm.loc[normalized_tm.index[i]][normalized_tm.loc[normalized_tm.index[i]].notna()].index.values
            for k in nodes:
                G.add_edge(normalized_tm.index[i],k,weight=normalized_tm.loc[normalized_tm.index[i]][k])  
#         plt.figure(figsize=(5,5))
        pos = nx.shell_layout(G)
        degree = dict(G.degree)
        edges = G.edges()
        weights = [G[u][v]['weight'] for u,v in edges]
#         nx.draw(G, pos, with_labels = True,node_size=[v * 100 for v in degree.values()], node_color="red")


        shortest_path_length_matrix = pd.DataFrame(index = G.nodes, columns = G.nodes)

        for i in G.nodes:
            for j in G.nodes:
                if np.isnan(normalized_tm.loc[i][j]):
                    continue
                if i==j:
                    shortest_path_length_matrix.loc[i][j] = G[i][j]['weight']
                elif i!=j:
                    shortest_path_length_matrix.loc[i][j] = nx.shortest_path_length(G,i,j,weight = 'weight')


        Dg = shortest_path_length_matrix.max().max()
        return shortest_path_length_matrix,G,Dg


# Historical data

In [14]:
def QueryTrajectory(testData,queryClient,points,n):
    print('Preparing the query trajectories')
    if points == 'exit':
        # Test client's trajectories as defined Tc = [(p1,(s1,e1)),(p2,(s2,e2)),..,(pN,(sN,eN))] --> list of tuples
        query_trajectories = {}
        for i in range(len(queryClient)):
            temp = testData[testData['ClientID']==queryClient[i]]
            T = []
            for j in range(len(temp)):
                T = T + [(temp['ProjectType'].iloc[j],(temp['EntryDate'].iloc[j],temp['ExitDate'].iloc[j]))]
                if j == len(temp)-1:
                    T = T + [(-3)]
            query_trajectories[queryClient[i]] = T   
    elif points == 'interim':
        # Test client's trajectories as defined Tc = [(p1,(s1,e1)),(p2,(s2,e2)),..,(pN,(sN,eN))] --> list of tuples
        query_trajectories = {}
        for i in range(len(queryClient)):
            temp = testData[testData['ClientID']==queryClient[i]]
            T = []
            for j in range(len(temp)):
                T = T + [(temp['ProjectType'].iloc[j],(temp['EntryDate'].iloc[j],temp['ExitDate'].iloc[j]))]
            query_trajectories[queryClient[i]] = T  
        
    # Separating the query history and target for prediction
    # query_chopped_client = query_chopped_trajectory.keys()
    query_past = {}
    query_target = {}
    for i in queryClient:
        temp = query_trajectories[i]
        if len(temp)>1:
            query_past[i] = temp[0:len(temp)-1]
            if type(temp[-1]) == tuple:
                query_target[i] = temp[-1][0]
            else: 
                query_target[i] = temp[-1]
    
    query_past_chopped = {}
    for i in query_past.keys():
        temp = query_past[i]
        if len(temp)>n:
            temp1 = [temp[i] for i in range(n-1,-1,-1)] 
            temp1.reverse()
        else:
            temp1 = temp
        query_past_chopped[i] = temp1
        sNminusn = temp1[0][1][0]
        eN = temp1[-1][1][1]
    
    return query_past_chopped,query_target 

def HistClientWithinTimeInterval(query_past,historicalClient,testData,n,method = 'others'):
    print('Extracting the historical clients within the time interval')
    # Determining the queries time interval (s_(N-n),e_N) and the histroical data within that time interval
    b = 0
    temp_hist = pd.DataFrame()
    next_target = pd.DataFrame()


    for i in query_past_chopped.keys():
        print(b)
        temp1 = query_past_chopped[i]
        sNminusn = temp1[0][1][0]
        eN = temp1[-1][1][1]
        if method == 'others':
            for j in range(len(historicalClient)):
                temp2 = testData[testData['ClientID']==historicalClient[j]]
                temp2_updated = pd.DataFrame()
                if temp2['EntryDate'].iloc[0] >= eN or temp2['ExitDate'].iloc[-1] <= sNminusn:
                    continue
                for k in temp2.index:
                    if temp2['EntryDate'].loc[k]>=sNminusn and temp2['ExitDate'].loc[k]<=eN:
                        temp2_updated = temp2_updated.append(temp2.loc[k])
                        if k == temp2.index[-1]:
                            temp2_updated = temp2_updated.append({'ClientID':historicalClient[j],'ProjectType':-3},ignore_index=True)
                    elif temp2['EntryDate'].loc[k]<=sNminusn and temp2['ExitDate'].loc[k]>=sNminusn and temp2['ExitDate'].loc[k]<=eN:
                        temp2.at[k,'EntryDate'] = sNminusn
                        temp2_updated = temp2_updated.append(temp2.loc[k])
                        if k == temp2.index[-1]:
                            temp2_updated = temp2_updated.append({'ClientID':historicalClient[j],'ProjectType':-3},ignore_index=True)
                    elif temp2['EntryDate'].loc[k]>=sNminusn and temp2['EntryDate'].loc[k]<=eN and temp2['ExitDate'].loc[k]>=eN:
                        temp2.at[k,'ExitDate'] = eN
                        temp2_updated = temp2_updated.append(temp2.loc[k])
                        if k == temp2.index[-1]:
                            temp2_updated = temp2_updated.append({'ClientID':historicalClient[j],'ProjectType':-3},ignore_index=True)
                    elif temp2['EntryDate'].loc[k]<=sNminusn and temp2['ExitDate'].loc[k]>=eN:
                        temp2.at[k,'EntryDate'] = sNminusn
                        temp2.at[k,'ExitDate'] = eN
                        temp2_updated = temp2_updated.append(temp2.loc[k])    
                        if k == temp2.index[-1]:
                            temp2_updated = temp2_updated.append({'ClientID':historicalClient[j],'ProjectType':-3},ignore_index=True)

                temp2_updated['query client'] = [i for a in range(len(temp2_updated))]
                temp_hist = temp_hist.append(temp2_updated)
         
        elif method == 'baseline 1':
            for j in range(len(historicalClient)):
                temp2 = testData[testData['ClientID']==historicalClient[j]].reset_index(drop=True)
                temp2_updated = pd.DataFrame()
                if temp2['EntryDate'].iloc[0] >= eN or temp2['ExitDate'].iloc[-1] <= sNminusn:
                    continue
                for k in range(len(temp2)):
                    if temp2['EntryDate'].iloc[k]>=sNminusn and temp2['ExitDate'].iloc[k]<=eN:
                        temp2_updated = temp2_updated.append(temp2.iloc[k])
                        if k == len(temp2)-1:
                            next_target = next_target.append({'query client':i,
                                                              'historical client':historicalClient[j],
                                                              "next_node":-3},ignore_index=True)
                        else:
                            next_target = next_target.append({'query client':i,
                                                              'historical client':historicalClient[j],
                                                              "next_node":temp2['ProjectType'].iloc[k+1]},ignore_index=True)


                    elif temp2['EntryDate'].iloc[k]<=sNminusn and temp2['ExitDate'].iloc[k]>=sNminusn and temp2['ExitDate'].iloc[k]<=eN:
                        temp2.at[k,'EntryDate'] = sNminusn
                        temp2_updated = temp2_updated.append(temp2.iloc[k])
                        if k == len(temp2)-1:
                            next_target = next_target.append({'query client':i,
                                                              'historical client':historicalClient[j],
                                                              "next_node":-3},ignore_index=True)
                        else:
                            next_target = next_target.append({'query client':i,
                                                              'historical client':historicalClient[j],
                                                              "next_node":temp2['ProjectType'].iloc[k+1]},ignore_index=True)

                    elif temp2['EntryDate'].iloc[k]>=sNminusn and temp2['EntryDate'].iloc[k]<=eN and temp2['ExitDate'].iloc[k]>=eN:
                        temp2.at[k,'ExitDate'] = eN
                        temp2_updated = temp2_updated.append(temp2.iloc[k])
                        if temp2['ExitDate'].iloc[k]>eN:
                            next_target = next_target.append({'query client':i,
                                                              'historical client':historicalClient[j],
                                                              "next_node":temp2['ProjectType'].iloc[k]},ignore_index=True)
                        elif temp2['ExitDate'].iloc[k] == eN:
                            if k == len(temp2)-1:
                                next_target = next_target.append({'query client':i,
                                                              'historical client':historicalClient[j],
                                                              "next_node":-3},ignore_index=True)
                            else:
                                next_target = next_target.append({'query client':i,
                                                              'historical client':historicalClient[j],
                                                              "next_node":temp2['ProjectType'].iloc[k+1]},ignore_index=True)

                    elif temp2['EntryDate'].iloc[k]<=sNminusn and temp2['ExitDate'].iloc[k]>=eN:
                        temp2.at[k,'EntryDate'] = sNminusn
                        temp2.at[k,'ExitDate'] = eN
                        temp2_updated = temp2_updated.append(temp2.iloc[k])   
                        if temp2['ExitDate'].iloc[k]>eN:
                            next_target = next_target.append({'query client':i,
                                                              'historical client':historicalClient[j],
                                                              "next_node":temp2['ProjectType'].iloc[k]},ignore_index=True)
                        elif temp2['ExitDate'].iloc[k] == eN:
                            if k == len(temp2)-1:
                                next_target = next_target.append({'query client':i,
                                                              'historical client':historicalClient[j],
                                                              "next_node":-3},ignore_index=True)
                            else:
                                next_target = next_target.append({'query client':i,
                                                              'historical client':historicalClient[j],
                                                              "next_node":temp2['ProjectType'].iloc[k+1]},ignore_index=True)
                temp2_updated['query client'] = [i for a in range(len(temp2_updated))]
                temp_hist = temp_hist.append(temp2_updated)
        b = b+1
    if method == 'others':
        return temp_hist
    elif method == 'baseline 1':
        return temp_hist,next_target

    


# Similarity computation

In [16]:
def Overlap(query_past_chopped,temp_hist):
    print("Computing overlap")
    # Computing overlap for each test query
    testingQueryClientUnique = temp_hist['query client'].unique()
    overlap = pd.DataFrame()
    for a in range(len(testingQueryClientUnique)):
        print(a)
        temp = temp_hist[temp_hist['query client']==testingQueryClientUnique[a]]
        historicalClientUnique = temp['ClientID'].unique()
        for b in range(len(historicalClientUnique)):
            temp1 = temp[temp['ClientID']==historicalClientUnique[b]].reset_index(drop=True)
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]

            for k in range(len(testingQueryTrajectory)-1,-1,-1):
                tq = testingQueryTrajectory[k][1]
                q = testingQueryTrajectory[k][0]
                for m in range(len(temp1)):
    #                 print(tq,temp1.iloc[m]['EntryDate'],temp1.iloc[m]['ExitDate'])
                    if temp1.iloc[m]['EntryDate']>=tq[0] and temp1.iloc[m]['ExitDate']<=tq[1]:

                        overlap_value = abs(temp1.iloc[m]['ExitDate']-temp1.iloc[m]['EntryDate'])
                        if temp1.iloc[m]['ExitDate']==temp1.iloc[m]['EntryDate']:
                            overlap_value = timedelta(days=1)

                        overlap = overlap.append({'query client':testingQueryClientUnique[a],
                                                  'historical client':historicalClientUnique[b],
                                                  "overlap initial":temp1.iloc[m]['EntryDate'],
                                                  "overlap final":temp1.iloc[m]['ExitDate'],
                                                  "overlap": overlap_value,
                                                  'q':q,
                                                  'node':temp1.iloc[m]['ProjectType']},ignore_index = True)
                    elif temp1.iloc[m]['EntryDate']<tq[0] and temp1.iloc[m]['ExitDate']>tq[0] and temp1.iloc[m]['ExitDate']<=tq[1]:

                        overlap_value = abs(temp1.iloc[m]['ExitDate']-tq[0])
                        if temp1.iloc[m]['ExitDate']==tq[0]:
                            overlap_value = timedelta(days=1)
                        overlap = overlap.append({'query client':testingQueryClientUnique[a],
                                                  'historical client':historicalClientUnique[b],
                                                  "overlap initial":tq[0],
                                                  "overlap final":temp1.iloc[m]['ExitDate'],
                                                  "overlap": overlap_value,
                                                  "q":q,
                                                  'node':temp1.iloc[m]['ProjectType']},ignore_index = True)            
                    elif temp1.iloc[m]['EntryDate']>=tq[0] and temp1.iloc[m]['EntryDate']<tq[1] and temp1.iloc[m]['ExitDate']>tq[1]:

                        overlap_value = abs(tq[1]-temp1.iloc[m]['EntryDate'])
                        if tq[1]==temp1.iloc[m]['EntryDate']:
                            overlap_value = timedelta(days=1)
                        overlap = overlap.append({'query client':testingQueryClientUnique[a],
                                                  'historical client':historicalClientUnique[b],
                                                  "overlap initial":temp1.iloc[m]['EntryDate'],
                                                  "overlap final":tq[1],
                                                  "overlap": overlap_value,
                                                  'q':q,
                                                  'node':temp1.iloc[m]['ProjectType']},ignore_index = True)                        
                    elif temp1.iloc[m]['EntryDate']<tq[0] and temp1.iloc[m]['ExitDate']>tq[1]:

                        overlap_value = abs(tq[1]-tq[0])
                        if tq[1]==tq[0]:
                            overlap_value = timedelta(days=1)
                        overlap = overlap.append({'query client':testingQueryClientUnique[a],
                                                  'historical client':historicalClientUnique[b],
                                                  "overlap initial":tq[0],
                                                  "overlap final":tq[1],
                                                  "overlap" : overlap_value,
                                                  'q':q,
                                                  'node':temp1.iloc[m]['ProjectType']},ignore_index = True)  

    overlap['overlap'] = overlap['overlap'].dt.days
    overlap = overlap.sort_values(["overlap initial",'overlap final']).reset_index(drop=True)
    return overlap

def SimilarityInfoComputation(temp_hist,overlap,method):
    print("Computing Similarity info")
    if method == 'method 1':
        testingQueryClientUnique = temp_hist['query client'].unique()
        similarity_info = pd.DataFrame()
        minDistanceData = pd.DataFrame()
        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp = overlap[overlap['query client']==testingQueryClientUnique[a]]
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
            trajectory_effective_length = len(pd.DataFrame([testingQueryTrajectory[n][0] for n in range(len(testingQueryTrajectory))])[0].unique())
            for k in range(len(testingQueryTrajectory)-1,-1,-1):
                tq = testingQueryTrajectory[k][1]
                q = testingQueryTrajectory[k][0]
                overlapping_interval = temp[(temp['overlap initial']>=tq[0]) & (temp['overlap final']<=tq[1])]
                overlapping_interval_hist_client = overlapping_interval['historical client'].unique()
                for l in range(len(overlapping_interval_hist_client)):
                    temp1 = overlapping_interval[overlapping_interval['historical client']==overlapping_interval_hist_client[l]]
                    maxOverlap = temp1.loc[temp1['overlap']==temp1['overlap'].max()]
                    maxOverlap = maxOverlap.reset_index(drop=True)
                    for i in range(len(maxOverlap)):
                        maxOverlap.at[i,'distance'] = shortest_path_length_list[trajectory_effective_length].loc[maxOverlap.iloc[i]['q']][maxOverlap.iloc[i]['node']]/Dg_list[trajectory_effective_length]
                    maxOverlap['distance'] = maxOverlap['distance'].fillna(10000)
                    minDistance = maxOverlap.loc[maxOverlap['distance']==maxOverlap['distance'].min()].iloc[-1]
                    minDistanceData = minDistanceData.append(minDistance)
        similarity_info = minDistanceData
        similarity_info = similarity_info.drop_duplicates(keep='first')
    
        return similarity_info
    elif method == 'method 2':
        # Determining the max overlap 
        testingQueryClientUnique = temp_hist['query client'].unique()
        similarity_info = pd.DataFrame()
        minDistanceData_l1 = pd.DataFrame()
        minDistanceData_l2 = pd.DataFrame()

        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp = overlap[overlap['query client']==testingQueryClientUnique[a]]
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
            trajectory_effective_length = len(pd.DataFrame([testingQueryTrajectory[n][0] for n in range(len(testingQueryTrajectory))])[0].unique())
            for k in range(len(testingQueryTrajectory)-1,-1,-1):
                tq = testingQueryTrajectory[k][1]
                q = testingQueryTrajectory[k][0]
                overlapping_interval = temp[(temp['overlap initial']>=tq[0]) & (temp['overlap final']<=tq[1])]
                overlapping_interval_hist_client = overlapping_interval['historical client'].unique()
                for l in range(len(overlapping_interval_hist_client)):
                    temp1 = overlapping_interval[overlapping_interval['historical client']==overlapping_interval_hist_client[l]]
                    maxOverlap = temp1.loc[temp1['overlap']==temp1['overlap'].max()]
                    maxOverlap = maxOverlap.reset_index(drop=True)
                    for i in range(len(maxOverlap)):
                        maxOverlap.at[i,'distance_l'] = shortest_path_length_list[trajectory_effective_length].loc[maxOverlap.iloc[i]['q']][maxOverlap.iloc[i]['node']]/Dg_list[trajectory_effective_length]
                        maxOverlap.at[i,'distance_l+1'] = shortest_path_length_list[trajectory_effective_length+1].loc[maxOverlap.iloc[i]['q']][maxOverlap.iloc[i]['node']]/Dg_list[trajectory_effective_length+1]

                    maxOverlap['distance_l'] = maxOverlap['distance_l'].fillna(10000)
                    maxOverlap['distance_l+1'] = maxOverlap['distance_l+1'].fillna(10000)

                    minDistance_l1 = maxOverlap.loc[maxOverlap['distance_l']==maxOverlap['distance_l'].min()].iloc[-1]
                    minDistanceData_l1 = minDistanceData_l1.append(minDistance_l1)

                    minDistance_l2 = maxOverlap.loc[maxOverlap['distance_l+1']==maxOverlap['distance_l+1'].min()].iloc[-1]
                    minDistanceData_l2 = minDistanceData_l2.append(minDistance_l2)

        similarity_info_l1 = minDistanceData_l1 
        similarity_info_l2 = minDistanceData_l2
        similarity_info_l1 = similarity_info_l1.drop_duplicates(keep='first')
        similarity_info_l2 = similarity_info_l2.drop_duplicates(keep='first')
        return similarity_info_l1,similarity_info_l2
    
    elif method == 'baseline':
        testingQueryClientUnique = temp_hist['query client'].unique()
        minDistanceData = pd.DataFrame()
        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp = overlap[overlap['query client']==testingQueryClientUnique[a]]
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
        #     trajectory_effective_length = len(pd.DataFrame([testingQueryTrajectory[n][0] for n in range(len(testingQueryTrajectory))])[0].unique())
            for k in range(len(testingQueryTrajectory)-1,-1,-1):
                tq = testingQueryTrajectory[k][1]
                q = testingQueryTrajectory[k][0]
                overlapping_interval = temp[(temp['overlap initial']>=tq[0]) & (temp['overlap final']<=tq[1])]
                overlapping_interval_hist_client = overlapping_interval['historical client'].unique()
                for l in range(len(overlapping_interval_hist_client)):
                    temp1 = overlapping_interval[overlapping_interval['historical client']==overlapping_interval_hist_client[l]]
                    maxOverlap = temp1.loc[temp1['overlap']==temp1['overlap'].max()]
                    maxOverlap = maxOverlap.reset_index(drop=True)
                    for i in range(len(maxOverlap)):
                        maxOverlap.at[i,'distance'] = shortest_path_length_matrix.loc[maxOverlap.iloc[i]['q']][maxOverlap.iloc[i]['node']]/Dg
                    maxOverlap['distance'] = maxOverlap['distance'].fillna(10000)
                    minDistance = maxOverlap.loc[maxOverlap['distance']==maxOverlap['distance'].min()].iloc[-1]
                    minDistanceData = minDistanceData.append(minDistance)
        similarity_info = minDistanceData
        return similarity_info





def SimilarityComputation(similarity_info,method):
    print('Computing similarity')
    if method == 'method 1':
        similarity_info = similarity_info[0]
        # Computing the similarity between trajectories
        testingQueryClientUnique = similarity_info['query client'].unique()
        similarity = pd.DataFrame()
        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp = similarity_info[similarity_info['query client']==testingQueryClientUnique[a]]
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
            historicalClientUnique = temp['historical client'].unique()
            similarity_denum = 0
            for k in range(len(testingQueryTrajectory)-1,-1,-1):
                tq = testingQueryTrajectory[k][1]
                if tq[0]==tq[1]:
                    overlapped = 1
                else: 
                    overlapped = (tq[1]-tq[0]).days
                similarity_denum = similarity_denum + overlapped

            for b in range(len(historicalClientUnique)):
                temp1 = temp[temp['historical client']==historicalClientUnique[b]].reset_index(drop=True)
                temp1 = temp1.sort_values("overlap initial").reset_index(drop=True)
                trac = list(temp1['node'])
                similarity_sum = 0
                for k in range(len(temp1)):
                    overlapPerTimeInterval = temp1.iloc[k]["overlap"]
                    d = temp1.iloc[k]["distance"]
                    similarity_sum = similarity_sum + (overlapPerTimeInterval*np.exp(-d))/similarity_denum

                similarity = similarity.append({"historical client":historicalClientUnique[b],
                                                "query client":testingQueryClientUnique[a],
                                               "historical trajectory":trac,
                                                "historical distance": list(temp1['distance'].values),
                                                "historical overlap": list(temp1['overlap']),
                                                "historical initial": list(temp1['overlap initial'].dt.date),
                                                "historical final": list(temp1['overlap final'].dt.date),
                                                "length_historical trajectory":len(trac),
                                                "length_query trajectory":len(testingQueryTrajectory),
                                                'overlap': temp1['overlap'].sum(),
                                               "similarity":similarity_sum},ignore_index=True)  
    elif method == 'method 2':
        similarity_info_l1 = similarity_info[0]
        similarity_info_l2 = similarity_info[1]
        # Computing the similarity between trajectories using minimum distance for l
        print('Computing similarity for l')
        testingQueryClientUnique = similarity_info_l1['query client'].unique()
        similarity_l1 = pd.DataFrame()
        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp = similarity_info_l1[similarity_info_l1['query client']==testingQueryClientUnique[a]]
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
            historicalClientUnique = temp['historical client'].unique()
            similarity_denum = 0
            for k in range(len(testingQueryTrajectory)-1,-1,-1):
                tq = testingQueryTrajectory[k][1]
                if tq[0]==tq[1]:
                    overlapped = 1
                else: 
                    overlapped = (tq[1]-tq[0]).days
                similarity_denum = similarity_denum + overlapped
            for b in range(len(historicalClientUnique)):
                temp1 = temp[temp['historical client']==historicalClientUnique[b]].reset_index(drop=True)
                temp1 = temp1.sort_values("overlap initial").reset_index(drop=True)
                trac = list(temp1['node'])
                similarity_sum = 0
                for k in range(len(temp1)):
                    overlapPerTimeInterval = temp1.iloc[k]["overlap"]
                    d = temp1.iloc[k]["distance_l"]
                    similarity_sum = similarity_sum + (overlapPerTimeInterval*np.exp(-d))/similarity_denum

                similarity_l1 = similarity_l1.append({"historical client":historicalClientUnique[b],
                                                "query client":testingQueryClientUnique[a],
                                               "historical trajectory":trac,
                                                "historical distance": list(temp1['distance_l'].values),
                                                "historical overlap": list(temp1['overlap']),
                                                "historical initial": list(temp1['overlap initial'].dt.date),
                                                "historical final": list(temp1['overlap final'].dt.date),
                                                "effective length_historical trajectory":len(trac),
                                                "length_query trajectory":len(testingQueryTrajectory),
                                                'overlap': temp1['overlap'].sum(),
                                               "similarity":similarity_sum},ignore_index=True)  
        

        # Computing the similarity between trajectories using minimum distance for l+1
        print('Computing similarity for l+1')
        testingQueryClientUnique = similarity_info_l2['query client'].unique()
        similarity_l2 = pd.DataFrame()
        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp = similarity_info_l2[similarity_info_l2['query client']==testingQueryClientUnique[a]]
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
            historicalClientUnique = temp['historical client'].unique()
            similarity_denum = 0
            for k in range(len(testingQueryTrajectory)-1,-1,-1):
                tq = testingQueryTrajectory[k][1]
                if tq[0]==tq[1]:
                    overlapped = 1
                else: 
                    overlapped = (tq[1]-tq[0]).days
                similarity_denum = similarity_denum + overlapped
            for b in range(len(historicalClientUnique)):
                temp1 = temp[temp['historical client']==historicalClientUnique[b]].reset_index(drop=True)
                temp1 = temp1.sort_values("overlap initial").reset_index(drop=True)
                trac = list(temp1['node'])
                similarity_sum = 0
                for k in range(len(temp1)):
                    overlapPerTimeInterval = temp1.iloc[k]["overlap"]
                    d = temp1.iloc[k]["distance_l+1"]
                    similarity_sum = similarity_sum + (overlapPerTimeInterval*np.exp(-d))/similarity_denum

                similarity_l2 = similarity_l2.append({"historical client":historicalClientUnique[b],
                                                "query client":testingQueryClientUnique[a],
                                               "historical trajectory":trac,
                                                "historical distance": list(temp1['distance_l+1'].values),
                                                "historical overlap": list(temp1['overlap']),
                                                "historical initial": list(temp1['overlap initial'].dt.date),
                                                "historical final": list(temp1['overlap final'].dt.date),
                                                "effective length_historical trajectory":len(trac),
                                                "length_query trajectory":len(testingQueryTrajectory),
                                                'overlap': temp1['overlap'].sum(),
                                               "similarity":similarity_sum},ignore_index=True)     
      

        similarity = [similarity_l1,similarity_l2]
    elif method == 'baseline':
        similarity_info = similarity_info[0]
        # Computing the similarity between trajectories
        testingQueryClientUnique = similarity_info['query client'].unique()
        similarity = pd.DataFrame()
        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp = similarity_info[similarity_info['query client']==testingQueryClientUnique[a]]
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
            historicalClientUnique = temp['historical client'].unique()
            similarity_denum = 0
            for k in range(len(testingQueryTrajectory)-1,-1,-1):
                tq = testingQueryTrajectory[k][1]
                if tq[0]==tq[1]:
                    overlapped = 1
                else: 
                    overlapped = (tq[1]-tq[0]).days
                similarity_denum = similarity_denum + overlapped

            for b in range(len(historicalClientUnique)):
                temp1 = temp[temp['historical client']==historicalClientUnique[b]].reset_index(drop=True)
                temp1 = temp1.sort_values("overlap initial").reset_index(drop=True)
                trac = list(temp1['node'])
                similarity_sum = 0
                for k in range(len(temp1)):
                    overlapPerTimeInterval = temp1.iloc[k]["overlap"]
                    d = temp1.iloc[k]["distance"]
                    similarity_sum = similarity_sum + (overlapPerTimeInterval*np.exp(-d))/similarity_denum

                similarity = similarity.append({"historical client":historicalClientUnique[b],
                                                "query client":testingQueryClientUnique[a],
                                               "historical trajectory":trac,
                                                "historical distance": list(temp1['distance'].values),
                                                "historical overlap": list(temp1['overlap']),
                                                "historical initial": list(temp1['overlap initial'].dt.date),
                                                "historical final": list(temp1['overlap final'].dt.date),
                                                "length_historical trajectory":len(trac),
                                                "length_query trajectory":len(testingQueryTrajectory),
                                                'overlap': temp1['overlap'].sum(),
                                               "similarity":similarity_sum},ignore_index=True)  

 
    elif method == 'baseline 6':
        similarity_info = similarity_info[0]
        # Computing the similarity between trajectories
        testingQueryClientUnique = similarity_info['query client'].unique()
        similarity = pd.DataFrame()
        beta = 0.8
        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp = similarity_info[similarity_info['query client']==testingQueryClientUnique[a]]
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
            historicalClientUnique = temp['historical client'].unique()
            similarity_denum = 0
            for k in range(len(testingQueryTrajectory)-1,-1,-1):
                tq = testingQueryTrajectory[k][1]
                if tq[0]==tq[1]:
                    overlapped = 1
                else: 
                    overlapped = (tq[1]-tq[0]).days
                similarity_denum = similarity_denum + overlapped

            for b in range(len(historicalClientUnique)):
                temp1 = temp[temp['historical client']==historicalClientUnique[b]].reset_index(drop=True)
                temp1 = temp1.sort_values("overlap initial").reset_index(drop=True)
                trac = list(temp1['node'])
                similarity_sum = 0
                for k in range(len(temp1)):
                    overlapPerTimeInterval = temp1.iloc[k]["overlap"]
                    d = temp1.iloc[k]["distance"]
                    similarity_sum = similarity_sum + (beta**(len(temp1)-k-1)*overlapPerTimeInterval*np.exp(-d))/similarity_denum
                similarity = similarity.append({"historical client":historicalClientUnique[b],
                                                "query client":testingQueryClientUnique[a],
                                               "historical trajectory":trac,
                                                "historical distance": list(temp1['distance'].values),
                                                "historical overlap": list(temp1['overlap']),
                                                "historical initial": list(temp1['overlap initial'].dt.date),
                                                "historical final": list(temp1['overlap final'].dt.date),
                                                "length_historical trajectory":len(trac),
                                                "length_query trajectory":len(testingQueryTrajectory),
                                                'overlap': temp1['overlap'].sum(),
                                               "similarity":similarity_sum},ignore_index=True)  
    
 
    return similarity
    

        
    
    
            

    

def Prediction(similarity,method,k=10):
    print('Predicting')
    if method == 'method 1':
        similarity = similarity[0]
        testingQueryClientUnique = similarity['query client'].unique()
        prediction = pd.DataFrame(index = range(len(testingQueryClientUnique)),columns=['ClientID','actual']+['Prediction@'+str(i+1) for i in range(k)])

        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp = similarity[similarity['query client']==testingQueryClientUnique[a]]
            temp = temp.sort_values(by=['similarity'],ascending=False)
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
            testingQueryTarget = query_target[testingQueryClientUnique[a]]
            testing_trajectory_effective_length = len(pd.DataFrame([testingQueryTrajectory[n][0] for n in range(len(testingQueryTrajectory))])[0].unique())
            qn = testingQueryTrajectory[-1][0]
            NB_q = set([n for n in G_list[testing_trajectory_effective_length].neighbors(qn)])
            prediction.at[a,'ClientID'] = testingQueryClientUnique[a]
            prediction.at[a,'actual'] = testingQueryTarget

            temp = temp.sort_values('similarity',ascending=False)
            maxSimilarity = pd.DataFrame()
            i = 0
            maxSimilarity = maxSimilarity.append(temp.iloc[i])
         

            for j in range(1,len(temp)):
                if i >=k:
                    break
                if temp['historical trajectory'].iloc[j] == maxSimilarity.iloc[i]['historical trajectory'] and temp['similarity'].iloc[j] == maxSimilarity.iloc[i]['similarity']:
                    continue
                else:
                    i = i+1
                    if i<k:
                        maxSimilarity = maxSimilarity.append(temp.iloc[j])
            maxSimilarity = maxSimilarity.reset_index(drop=True)
            maxSimilarity = maxSimilarity.sort_values('similarity',ascending=False)
            
            for i in range(k):
                if i>=len(maxSimilarity):
                    continue
                similar_historical_trajectory = maxSimilarity.iloc[i]['historical trajectory']
                similar_trajectory_effective_length = len(pd.DataFrame(similar_historical_trajectory)[0].unique())
                pn = similar_historical_trajectory[-1]
                NB_p = set([m for m in G_list[similar_trajectory_effective_length].neighbors(pn)])
                NBp_NBq_intersect = NB_p.intersection(NB_q)
                pntoNBp_NBq_intersect = {}
                for b in NBp_NBq_intersect:
                    pntoNBp_NBq_intersect[('p',b)] = G_list[similar_trajectory_effective_length][pn][b]["weight"]
                if not pntoNBp_NBq_intersect:
                    continue
                maxProbNode = max(pntoNBp_NBq_intersect,key=pntoNBp_NBq_intersect.get)
                
                testingQueryPred = maxProbNode[1]
                prediction.at[a,'Prediction@'+str(i+1)] = testingQueryPred
    elif method == 'method 2':
        
        similarity_l1 = similarity[0]
        similarity_l2 = similarity[1]
        # Computing the prediction of test queries using two of the similarities
        testingQueryClientUnique = similarity_l1['query client'].unique()
        prediction = pd.DataFrame(index = range(len(testingQueryClientUnique)),columns=['ClientID','actual']+['Prediction@'+str(i+1) for i in range(k)])
        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp1 = similarity_l1[similarity_l1['query client']==testingQueryClientUnique[a]]
            temp1 = temp1.sort_values(by=['similarity'],ascending=False)
            temp2 = similarity_l2[similarity_l2['query client']==testingQueryClientUnique[a]]
            temp2 = temp2.sort_values(by=['similarity'],ascending=False)
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
            testingQueryTarget = query_target[testingQueryClientUnique[a]]
            testing_trajectory_effective_length = len(pd.DataFrame([testingQueryTrajectory[n][0] for n in range(len(testingQueryTrajectory))])[0].unique())
            qn = testingQueryTrajectory[-1][0]
            NB_q = set([n for n in G_list[testing_trajectory_effective_length].neighbors(qn)])
            prediction.at[a,'ClientID'] = testingQueryClientUnique[a]
            prediction.at[a,'actual'] = testingQueryTarget

            temp1 = temp1.sort_values('similarity',ascending=False)
            maxSimilarity_l1 = pd.DataFrame()
            i = 0
            maxSimilarity_l1 = maxSimilarity_l1.append(temp1.iloc[i])
            for j in range(1,len(temp1)):
                if i >=k:
                    break
                if temp1['historical trajectory'].iloc[j] == maxSimilarity_l1.iloc[i]['historical trajectory'] and temp1['similarity'].iloc[j] == maxSimilarity_l1.iloc[i]['similarity']:
                    continue
                else:
                    i = i+1
                    if i<k:
                        maxSimilarity_l1 = maxSimilarity_l1.append(temp1.iloc[j])
            maxSimilarity_l1 = maxSimilarity_l1.reset_index(drop=True)
            maxSimilarity_l1 = maxSimilarity_l1.sort_values('similarity',ascending=False)

            temp2 = temp2.sort_values('similarity',ascending=False)
            maxSimilarity_l2 = pd.DataFrame()
            i = 0
            maxSimilarity_l2 = maxSimilarity_l2.append(temp2.iloc[i])
            k = 10
            for j in range(1,len(temp2)):
                if i >=k:
                    break
                if temp2['historical trajectory'].iloc[j] == maxSimilarity_l2.iloc[i]['historical trajectory'] and temp2['similarity'].iloc[j] == maxSimilarity_l2.iloc[i]['similarity']:
                    continue
                else:
                    i = i+1
                    if i<k:
                        maxSimilarity_l2 = maxSimilarity_l2.append(temp2.iloc[j])
            maxSimilarity_l2 = maxSimilarity_l2.reset_index(drop=True)
            maxSimilarity_l2 = maxSimilarity_l2.sort_values('similarity',ascending=False)


            maxSimilarity = pd.DataFrame()
            maxSimilarity = maxSimilarity.append(maxSimilarity_l1)
            maxSimilarity = maxSimilarity.append(maxSimilarity_l2)
            maxSimilarity = maxSimilarity.sort_values(by=['similarity'], ascending=False)

            for i in range(k):
                if i>=len(maxSimilarity):
                    continue
                similar_historical_trajectory = maxSimilarity.iloc[i]['historical trajectory']
                similar_trajectory_effective_length = len(pd.DataFrame(similar_historical_trajectory)[0].unique())
                pn = similar_historical_trajectory[-1]
                NB_p = set([m for m in G_list[similar_trajectory_effective_length].neighbors(pn)])
                NBp_NBq_intersect = NB_p.intersection(NB_q)
                pntoNBp_NBq_intersect = {}
                for b in NBp_NBq_intersect:
                    pntoNBp_NBq_intersect[('p',b)] = G_list[similar_trajectory_effective_length][pn][b]["weight"]
                if not pntoNBp_NBq_intersect:
                    continue
                maxProbNode = max(pntoNBp_NBq_intersect,key=pntoNBp_NBq_intersect.get)
                testingQueryPred = maxProbNode[1]
                prediction.at[a,'Prediction@'+str(i+1)] = testingQueryPred

    else:
        similarity = similarity[0]
        testingQueryClientUnique = similarity['query client'].unique()
        prediction = pd.DataFrame(index = range(len(testingQueryClientUnique)),columns=['ClientID','actual']+['Prediction@'+str(i+1) for i in range(k)])
        for a in range(len(testingQueryClientUnique)):
            print(a)
            temp = similarity[similarity['query client']==testingQueryClientUnique[a]]
            temp = temp.sort_values(by=['similarity'],ascending=False)
            testingQueryTrajectory = query_past_chopped[testingQueryClientUnique[a]]
            testingQueryTarget = query_target[testingQueryClientUnique[a]]
            testing_trajectory_effective_length = len(pd.DataFrame([testingQueryTrajectory[n][0] for n in range(len(testingQueryTrajectory))])[0].unique())
            qn = testingQueryTrajectory[-1][0]
            NB_q = set([n for n in G.neighbors(qn)])
            prediction.at[a,'ClientID'] = testingQueryClientUnique[a]
            prediction.at[a,'actual'] = testingQueryTarget

            temp = temp.sort_values('similarity',ascending=False)
            maxSimilarity = pd.DataFrame()
            i = 0
            maxSimilarity = maxSimilarity.append(temp.iloc[i])
            k = 10

            for j in range(1,len(temp)):
                if i >=k:
                    break
                if temp['historical trajectory'].iloc[j] == maxSimilarity.iloc[i]['historical trajectory'] and temp['similarity'].iloc[j] == maxSimilarity.iloc[i]['similarity']:
                    continue
                else:
                    i = i+1
                    if i<k:
                        maxSimilarity = maxSimilarity.append(temp.iloc[j])
            maxSimilarity = maxSimilarity.reset_index(drop=True)
            maxSimilarity = maxSimilarity.sort_values('similarity',ascending=False)

            for i in range(k):
                if i>=len(maxSimilarity):
                    continue
                if method == 'baseline 1':
                    testingQueryPred = next_target[(next_target['query client']==testingQueryClientUnique[a])&(next_target['historical client']==maxSimilarity.iloc[i]['historical client'])]['next_node'].values[0]
                    prediction.at[a,'Prediction@'+str(i+1)] = testingQueryPred
                elif method == 'baseline 2':
                    similar_historical_trajectory = maxSimilarity.iloc[i]['historical trajectory']
                    similar_trajectory_effective_length = len(pd.DataFrame(similar_historical_trajectory)[0].unique())
                    pn = similar_historical_trajectory[-1]
                    pntoNBp = {}
                    for m in G.neighbors(pn):
                        pntoNBp[('p',m)] = G[pn][m]["weight"]
                    maxProbNode = max(pntoNBp,key=pntoNBp.get)
                    testingQueryPred = maxProbNode[1]
                    prediction.at[a,'Prediction@'+str(i+1)] = testingQueryPred
                elif method == 'baseline 3':
                    similar_historical_trajectory = maxSimilarity.iloc[i]['historical trajectory']
                    similar_trajectory_effective_length = len(pd.DataFrame(similar_historical_trajectory)[0].unique())
                    pn = similar_historical_trajectory[-1]
                    qntoNBq = {}
                    for m in G.neighbors(qn):
                        qntoNBq[('q',m)] = G[qn][m]["weight"]
                    maxProbNode = max(qntoNBq,key=qntoNBq.get)
                    testingQueryPred = maxProbNode[1]
                    prediction.at[a,'Prediction@'+str(i+1)] = testingQueryPred
                elif method == 'baseline 4a':
                    testingQueryPred = np.random.choice(G.nodes)
                    prediction.at[a,'Prediction@'+str(i+1)] = testingQueryPred
                elif method == 'baseline 4b':
                    prob = [G.in_degree(node) for node in G.nodes]/np.sum([G.in_degree(node) for node in G.nodes])
                    testingQueryPred = np.random.choice(G.nodes,p=prob)
                    prediction.at[a,'Prediction@'+str(i+1)] = testingQueryPred
                elif method == 'baseline 5' or method == 'baseline 6':
                    similar_historical_trajectory = maxSimilarity.iloc[i]['historical trajectory']
                    similar_trajectory_effective_length = len(pd.DataFrame(similar_historical_trajectory)[0].unique())
                    pn = similar_historical_trajectory[-1]
                    NB_p = set([m for m in G.neighbors(pn)])
                    NBp_NBq_intersect = NB_p.intersection(NB_q)
                    pntoNBp_NBq_intersect = {}
                    for b in NBp_NBq_intersect:
                        pntoNBp_NBq_intersect[('p',b)] = G[pn][b]["weight"]
                    if not pntoNBp_NBq_intersect:
                        continue
                    maxProbNode = max(pntoNBp_NBq_intersect,key=pntoNBp_NBq_intersect.get)
                    testingQueryPred = maxProbNode[1]
                    prediction.at[a,'Prediction@'+str(i+1)] = testingQueryPred
                


    return prediction

# Evaluation

In [18]:
def Precision(prediction,k=10):
    precision_list = pd.DataFrame(index=range(1,k+1),columns=['precision@k'])
    precision = 0
    actual = prediction['actual']
    temp_prediction = prediction
    for j in range(k):
        precision = precision + len(temp_prediction[temp_prediction['Prediction@'+str(j+1)]==temp_prediction['actual']])
        precision_list.at[j+1,'precision@k'] = precision
        temp = temp_prediction[temp_prediction['Prediction@'+str(j+1)]!=temp_prediction['actual']]
        temp_prediction = temp
    return precision_list



def Recall(prediction,method,k=[2,6]):
    recall = pd.DataFrame(index= range(2*len(prediction.actual.unique())),columns = ['p','TP',"TP+FN"])
    i = 0
    for a in range(len(k)):
        pred = prediction[['Prediction@'+str(j+1) for j in range(k[a])]]
        pred['actual'] = prediction['actual']
        for p in prediction.actual.unique():
            TP = 0
            for col in ['Prediction@'+str(j+1) for j in range(k[a])]:    
                temp = pred[(pred[col]==p)&(pred['actual']==p)]
                index = temp.index
                TP = TP + len(temp)
                pred = pred.drop(index)



            recall.at[i,'k'] = k[a]
            recall.at[i,'p'] = p
            recall.at[i,'TP'] = TP
            recall.at[i,'TP+FN'] = len(prediction[prediction['actual']==p])
            i = i+1
    recall['recall'] = recall['TP']/recall['TP+FN']
    recall['method'] = [method for i in range(len(recall))]
    return recall


# Plotting 

In [12]:
def precision_plot(precision_list,m):
    linewidth = 2
    if m=='Method 1':
        marker = 'o'
        color = 'red'
        linestyle = 'solid'
        return plt.plot(precision_list.index,precision_list['precision@k']/222,marker=marker,color=color,linestyle=linestyle,linewidth = linewidth,label=m)

    elif m == 'Method 2':
        marker = 'o'
        color = 'red'
        linestyle = 'dotted'
        plt.plot(precision_list.index,precision_list['precision@k']/222,marker=marker,color=color,linestyle=linestyle,linewidth = linewidth,label=m)
    
    elif m == 'Baseline 1':
        marker = '^'
        color = 'green'
        linestyle = '--'
        plt.plot(precision_list.index,precision_list['precision@k']/222,marker=marker,color=color,linestyle=linestyle,linewidth = linewidth,label=m)
    
    elif m == 'Baseline 2':
        marker = 'D'
        color = 'black'
        linestyle = 'dotted'
        plt.plot(precision_list.index,precision_list['precision@k']/222,marker=marker,color=color,linestyle=linestyle,linewidth = linewidth,label=m)


    elif m == 'Baseline 5':
        marker = '*'
        color = 'brown'
        linestyle = 'dotted'
        plt.plot(precision_list.index,precision_list['precision@k']/222,marker=marker,color=color,linestyle=linestyle,linewidth = linewidth,label=m)
    
    elif m == 'Baseline 3':
        marker = 'h'
        color = 'orange'
        linestyle = '--'
        plt.plot(precision_list.index,precision_list['precision@k']/222,marker=marker,color=color,linestyle=linestyle,linewidth = linewidth,label=m)
    
    elif m == 'Baseline 4a':
        marker = 'D'
        color = 'purple'
        linestyle = '-.'
        plt.plot(precision_list.index,precision_list['precision@k']/222,marker=marker,color=color,linestyle=linestyle,linewidth = linewidth,label=m)
    
    elif m == 'Baseline 4b':
        marker = 'x'
        color = 'purple'
        linestyle = '-.'
        plt.plot(precision_list.index,precision_list['precision@k']/222,marker=marker,color=color,linestyle=linestyle,linewidth = linewidth,label=m)
   
    elif m == 'Baseline 6':
        marker = '8'
        color = 'blue'
        linestyle = ':'
        plt.plot(precision_list.index,precision_list['precision@k']/222,marker=marker,color=color,linestyle=linestyle,linewidth = linewidth,label=m)
    